In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])
dropProb = tf.placeholder(tf.float32)
phase = tf.placeholder(tf.bool, name='phase')


W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01))
# L1 = tf.layers.batch_normalization(W1, training=phase)
L1 = tf.nn.relu(tf.matmul(X, W1))
L1 = tf.nn.dropout(L1, dropProb)

W2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
# L2 = tf.layers.batch_normalization(W2, training=phase)
L2 = tf.nn.relu(tf.matmul(L1, W2))
L2 = tf.nn.dropout(L2, dropProb)

W3 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L2, W3)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)


init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)



for epoch in range(30):
    total_cost = 0
    
    for i in range(total_batch):
        x_batch, y_batch = mnist.train.next_batch(batch_size)
#         _, cost_val = sess.run([optimizer ,cost], feed_dict={X: x_batch, Y: y_batch, dropProb: 0.8, phase: True})
        _, cost_val = sess.run([optimizer ,cost], feed_dict={X: x_batch, Y: y_batch, dropProb: 0.8})
#         _, cost_val = sess.run([optimizer ,cost], feed_dict={X: x_batch, Y: y_batch, phase: True})
        
        total_cost += cost_val

    print("epoch : %04d" % (epoch + 1), "Avg. cost = %.3f" % (total_cost/total_batch))
        
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print("Accuracy : " , (sess.run(accuracy, feed_dict = {X: mnist.test.images, Y: mnist.test.labels, dropProb: 1., phase: False})))
print("Accuracy : " , (sess.run(accuracy, feed_dict = {X: mnist.test.images, Y: mnist.test.labels, dropProb: 1.})))
# print("Accuracy : " , (sess.run(accuracy, feed_dict = {X: mnist.test.images, Y: mnist.test.labels, phase: False})))
        